In [1]:
def recommandation_finale(tconst):

    def recommandation(tconst):
        import pandas as pd
        from sklearn.neighbors import NearestNeighbors
        from sklearn.preprocessing import MinMaxScaler

        # Chargement des données
        df_ml = pd.read_csv("../machine learning/DF_ML.csv.gz")

        # On récupère les valeurs genre et pays qui correspondent au film sélectionné
        df_selection = df_ml[df_ml['tconst'] == tconst]
        colonnes_genre = [
            'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History',
            'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
            'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'
        ]
        colonnes_pays = [
            'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN',
            'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR',
            'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN',
            'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT',
            'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO',
            'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'
        ]

        genre = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_genre]
        pays = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_pays]

        index = df_ml.index
        df_ml_num = df_ml.select_dtypes('number')
        df_ml_cat = df_ml.select_dtypes(['object', 'category', 'string', 'bool'])

        # Normalisation des colonnes numériques
        SN = MinMaxScaler()
        df_ml_num_SN = pd.DataFrame(SN.fit_transform(df_ml_num), columns=df_ml_num.columns, index=index)

        df_ml_encoded = pd.concat([df_ml_num_SN, df_ml_cat], axis=1)

        # On sépare notre df en deux groupes, en fonction de la note
        bons_films = df_ml_encoded[df_ml_encoded['notes'] >= 0.7]
        
        # On crée une liste des colonnes à utiliser pour le modèle
        caracteristiques = df_ml_encoded.columns.drop(['tconst', 'nconst', 'title', 'title_ratings_numVotes', 'rating', 
            'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History',
            'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
            'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 
            'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN',
            'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR',
            'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN',
            'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT',
            'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO',
            'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

        # On veut que nos recommandations aient automatiquement un genre en commun et un pays de prod en commun
        bons_films = bons_films[bons_films[genre].any(axis=1)]
        bons_films = bons_films[bons_films[pays].any(axis=1)]

        # On crée notre modèle
        model = NearestNeighbors(n_neighbors=10, metric='euclidean')
        model.fit(bons_films[caracteristiques])

        # On déclare les caractéristiques du film sélectionné par l'utilisateur
        caract_film = df_ml_encoded[df_ml_encoded['tconst'] == tconst][caracteristiques]

        # On calcule les distances et indices des voisins
        distances, indices = model.kneighbors(caract_film)

        # On affiche la sélection des films en fonction des indices trouvés par le modèle
        if caract_film['notes'].values[0] > 0.7:
            distances = distances[0][1:6]
            indices = indices[0][1:6]
            selection = bons_films.iloc[indices]['tconst']
        else:
            distances = distances[0][0:5]
            indices = indices[0][0:5]
            selection = bons_films.iloc[indices]['tconst']

        selection = pd.DataFrame(selection).reset_index(drop=True)

        return selection

    def recommandation2(tconst):

        df_ml = pd.read_csv("../machine learning/DF_ML.csv.gz")

        #On récupère les valeurs genre et pays qui correspondent au film selectionné
        df_selection = df_ml[df_ml['tconst'] == tconst]
        colonnes_genre = ['Action', 'Adventure',
        'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
        'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
        'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
        'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']

        colonnes_pays = ['tmdb_US',
            'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
            'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
            'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
            'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
            'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
            'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA']

        genre = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_genre]
        pays = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_pays]

        index = df_ml.index
        df_ml_num = df_ml.select_dtypes('number')
        df_ml_cat = df_ml.select_dtypes(['object', 'category', 'string', 'bool'])

        from sklearn.preprocessing import MinMaxScaler
        SN = MinMaxScaler()
        df_ml_num_SN = pd.DataFrame(SN.fit_transform(df_ml_num), columns=df_ml_num.columns, index=index)

        df_ml_encoded = pd.concat([df_ml_num_SN, df_ml_cat], axis=1)

        #On crée une liste des colonnes à utiliser pour le modèle
        caracteristiques = df_ml_encoded.columns.drop(['tconst', 'nconst', 'title', 'title_ratings_numVotes', 'rating',
            'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History',
            'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
            'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 
            'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN',
            'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR',
            'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN',
            'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT',
            'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO',
            'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

        #On sépare notre df en deux groupes, en fonction de la note
        bons_films = df_ml_encoded[df_ml_encoded['notes'] >= 0.7]

        #On veut que nos recommandations aient automatiquement un genre en commun et un pays de prod en commun
        bons_films = bons_films[bons_films[genre].any(axis=1)]
        bons_films = bons_films[~bons_films[pays].any(axis=1)]

        #On crée notre modèle
        model = NearestNeighbors(n_neighbors=10, metric='euclidean')
        model.fit(bons_films[caracteristiques])

        #On déclare les caractéristiques du film sélectionné par l'utilisateur
        caract_film = df_ml_encoded[df_ml_encoded['tconst'] == tconst]
        caract_film = caract_film[caracteristiques]
        caract_film

        distances, indices = model.kneighbors(caract_film)

        #On affiche la selection des films en fonction des indices trouvés par le modèle
        if caract_film['notes'].values[0] > 0.7:
            distances = distances[0][1:6]
            indices = indices[0][1:6]
            selection = bons_films.iloc[indices]['tconst']
        else:
            distances = distances[0][0:5]
            indices = indices[0][0:5]
            selection = bons_films.iloc[indices]['tconst']

        selection = pd.DataFrame(selection).reset_index(drop=True)

        return selection2

    return afficher_resultats_similarite(selection, selection2)


